# 梯度提升树方法
尝试几种树模型: LGBM, XGBoost, GBDT

In [1]:
# 读取仅经过数据规整的数据. 类别特征进行了Labelencoding, 连续特征未进行处理, 时间特征处理成年龄
import pickle
import pandas as pd
pd.set_option('max_columns', 200)
from utils import submit, evaluation

with open(r'./data/train_T.pkl', 'rb') as f:
    df_train = pickle.load(f)
with open(r'./data/test_T.pkl', 'rb') as f:
    df_test = pickle.load(f)

In [2]:
df_train.head(10)

,id,XINGBIE,HYZK,ZHIYE,ZHICHEN,ZHIWU,XUELI,DWJJLX,DWSSHY,GRZHZT,GRJCJS,GRZHYE,GRZHSNJZYE,GRZHDNGJYE,GRYJCE,DWYJCE,DKFFE,DKYE,DKLL,age,label
0,train_0,1,3,6,3,0,1,9,12,0,1737.0,3223.515,801.310,837.000,312.00,312.00,175237,154112.935,2.708,18,0.0
1,train_1,2,3,6,3,0,1,1,0,0,4894.0,18055.195,53213.220,1065.200,795.84,795.84,300237,298252.945,2.979,34,0.0
2,train_2,1,3,6,3,0,1,9,9,0,10297.0,27426.600,13963.140,7230.020,1444.20,1444.20,150237,147339.130,2.708,27,0.0
3,train_3,1,3,6,3,0,1,9,7,0,10071.5,111871.130,99701.265,2271.295,1417.14,1417.14,350237,300653.780,2.708,37,0.0
4,train_4,2,3,6,3,0,1,27,14,0,2007.0,237.000,11028.875,35.780,325.50,325.50,150237,145185.010,2.708,33,0.0
5,train_5,1,3,6,3,0,1,9,14,0,1192.0,9648.315,7388.550,771.800,303.85,303.85,150237,149743.210,2.708,35,0.0
6,train_6,1,3,6,3,0,1,11,2,0,7297.0,22846.975,12179.565,5885.000,801.80,801.80,150237,146723.725,2.708,33,0.0
7,train_7,1,3,6,3,0,1,9,7,0,1399.0,1808.135,6405.055,1050.400,318.34,318.34,125237,64938.955,2.708,16,0.0
8,train_8,2,3,6,3,0,1,9,9,0,6408.0,79304.635,52365.820,6161.160,977.52,977.52,300237,270579.955,2.979,31,0.0
9,train_9,2,3,6,3,0,1,9,14,0,3573.5,2009.000,61126.175,536.750,637.38,637.38,230237,142296.640,2.708,36,0.0


In [3]:
cate_feat = ['XINGBIE', 'HYZK', 'ZHIYE', 'ZHICHEN', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
all_feat = list(df_train.columns)
all_feat.remove('id')
all_feat.remove('label')
cont_feat = all_feat
for item in cate_feat:
    cont_feat.remove(item)
cont_feat

['ZHIWU',
 'GRJCJS',
 'GRZHYE',
 'GRZHSNJZYE',
 'GRZHDNGJYE',
 'GRYJCE',
 'DWYJCE',
 'DKFFE',
 'DKYE',
 'DKLL',
 'age']

In [4]:
# 构建特征和标签
y_train = df_train['label']
X_train = df_train.drop(['id', 'label'], axis=1)
X_test = df_test.drop(['id'], axis=1)

# 使用LGBM进行建模
特征就使用最原始的特征. 训练和验证方式K-折交叉验证

In [8]:
import lightgbm as lgb
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

KF = StratifiedKFold(n_splits=5, random_state=723)
params = {
    'verbose':-1, 
    'objective':'binary',
    'metric':'auc',
    'num_iterations': 10000, 
    'is_unbalance': True,
}


oof_lgb = np.zeros(len(X_train))
predictions_lgb = np.zeros((len(X_test)))

for fold_, (trn_idx, val_idx) in enumerate(KF.split(X_train.values, y_train.values)):
    print("fold n°{}".format(fold_))
    print('trn_idx:',trn_idx)
    print('val_idx:',val_idx)
    trn_data = lgb.Dataset(X_train.iloc[trn_idx],label=y_train.iloc[trn_idx])    
    val_data = lgb.Dataset(X_train.iloc[val_idx],label=y_train.iloc[val_idx])
    num_round = 10000
    clf = lgb.train(
        params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds=200,  
        categorical_feature=cate_feat
    )
        
    oof_lgb[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(X_test, num_iteration=clf.best_iteration) 
print("AUC score: {}".format(roc_auc_score(y_train, oof_lgb)))
print("TPR weight: {}".format(evaluation(y_train, oof_lgb)))

fold n°0
trn_idx: [ 7984  7985  7986 ... 39997 39998 39999]
val_idx: [   0    1    2 ... 8249 8282 8288]
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[174]	training's auc: 0.994789	valid_1's auc: 0.941945
fold n°1
trn_idx: [    0     1     2 ... 39997 39998 39999]
val_idx: [ 7984  7985  7986 ... 15997 15998 15999]
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[247]	training's auc: 0.998126	valid_1's auc: 0.939837
fold n°2
trn_idx: [    0     1     2 ... 39997 39998 39999]
val_idx: [16000 16001 16002 ... 24382 24393 24455]
Training until validation scores don't improve for 200 rounds
[500]	training's auc: 0.999995	valid_1's auc: 0.922317
Early stopping, best iteration is:
[300]	training's auc: 0.999459	valid_1's auc: 0.925363
fold n°3
trn_idx: [    0     1     2 ... 39997 39998 39999]
val_idx: [23964 23965 23966 ... 32011 32014 32019]
Training until validation scores don't improve for 

In [9]:
# 对测试集进行预测, 并得到提交文件
res = predictions_lgb / 5
submit(res)

In [7]:
predictions_lgb.shape

(15000,)